In [1]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import gamma, skew
import casadi as ca

### create random samples

In [2]:

sample_size=10000

R1 = gamma.rvs(40, scale=1/8, size=sample_size)
R2 = gamma.rvs(50, scale=3/25, size=sample_size)
R3 = gamma.rvs(30, scale=47/300, size=sample_size)
D1 = gamma.rvs(14/5, scale=8/10, size=sample_size)
D2 = gamma.rvs(2, scale=3, size=sample_size)
D3 = gamma.rvs(np.sqrt(5), scale=np.sqrt(5), size=sample_size)
D4 = gamma.rvs(4, scale=11/8, size=sample_size)

In [3]:
def system_vibration(R,D,f):
    return np.sqrt(R**2 + (f*D)**2 - 1.8*f*R*D)

def analyze_system_vibration(S):
    print(f'mean: {np.round(S.mean(), 2)}, skewness: {np.round(skew(S), 2)}, std: {np.round(S.std(), 2)}')
    #plt.hist(system_vibration(R1,D1,1),bins=100);

### try inner optimization problem

given combination of R&D --> optimize f

In [4]:
def mean_casadi(data,sample_size):
    return ca.sum1(data)/sample_size

In [5]:
def calc_skew_casadi(data,sample_size):
    """
    Calculate skewness for a 1D array using g1 = m3 / m2**(3/2)
    Parameters:
        data: array-like (1D)
    Returns:
        skewness value (float)
    """
    mean = mean_casadi(data,sample_size)
    m2 = mean_casadi((data - mean)**2,sample_size)
    m3 = mean_casadi((data - mean)**3,sample_size)
    return m3 / m2**1.5

In [6]:
def inner_op(R,D,obj,init_f=0.5,verbose=False,alpha=0.5):
    opti = ca.Opti()
    f = opti.variable()
    S = system_vibration(R,D,f)
    S_mean = mean_casadi(S,sample_size)# ca.sum1(S)/sample_size
    if obj=='min_mean':
        obj = S_mean
    elif obj=='target_mean':
        obj = (S_mean-3.2)**2
    elif obj=='min_mean_and_skew':
        #S_skew = skew(system_vibration(R,D,f)) # no skew function in casadi implemented up to my knowledge...
        S_skew = calc_skew_casadi(S,sample_size)
        obj = alpha*S_mean + (1-alpha)*S_skew
    opti.minimize(obj)
    opti.subject_to(S_mean<3.8)
    #opti.subject_to(S_mean>3.0)
    opti.subject_to(f>=0)
    opti.subject_to(f<=1)
    if obj=='min_mean_and_skew':
        opti.subject_to(S_skew>0)
        opti.subject_to(S_skew<1.0)
    opti.set_initial(f, init_f)

    p_opts = {"expand":False,'verbose':0}
    s_opts = {"print_level": 0}
    opti.solver('fatrop',p_opts, s_opts)
    sol = opti.solve()

    if verbose:
        print('solution: f=',sol.value(f))
        print('maximized objective = ',sol.value(obj))
        print('mean(S) = ',sol.value(S_mean))
        print('skew(S) = ',skew(system_vibration(R,D,sol.value(f))))
    return sol.value(f), sol.value(S_mean), skew(system_vibration(R,D,sol.value(f)))

In [7]:
_=inner_op(R1,D2,'target_mean',verbose=True)

      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |   1.00ms ( 50.00us)   1.32ms ( 65.80us)        20
       nlp_g  |   1.00ms (142.86us) 470.00us ( 67.14us)         7
  nlp_grad_f  |        0 (       0) 878.00us (109.75us)         8
  nlp_hess_l  |   1.00ms (166.67us)   1.40ms (232.67us)         6
   nlp_jac_g  |   3.00ms (375.00us)   1.47ms (183.50us)         8
       total  |   6.00ms (  6.00ms)   6.17ms (  6.17ms)         1
solution: f= 0.5356211113155136
maximized objective =  0.009548930179345723
mean(S) =  3.297718627596512
skew(S) =  1.640436698127766


In [8]:
_=inner_op(R1,D2,'min_mean',verbose=True)

      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |   2.00ms (100.00us)   1.42ms ( 71.10us)        20
       nlp_g  |   2.00ms (285.71us) 966.00us (138.00us)         7
  nlp_grad_f  |   2.00ms (250.00us) 771.00us ( 96.37us)         8
  nlp_hess_l  |        0 (       0)   1.11ms (184.83us)         6
   nlp_jac_g  |        0 (       0) 967.00us (120.87us)         8
       total  |   6.00ms (  6.00ms)   5.83ms (  5.83ms)         1
solution: f= 0.5356211114237108
maximized objective =  3.2977186275965003
mean(S) =  3.2977186275965003
skew(S) =  1.6404366992374195


In [9]:
_=inner_op(R1,D2,'min_mean_and_skew',verbose=True)

      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |  11.00ms (379.31us)  10.42ms (359.34us)        29
       nlp_g  |   1.00ms (100.00us) 683.00us ( 68.30us)        10
  nlp_grad_f  |   4.00ms (363.64us)   4.94ms (448.82us)        11
  nlp_hess_l  |   7.00ms (777.78us)   7.30ms (811.22us)         9
   nlp_jac_g  |   2.00ms (181.82us)   1.30ms (118.55us)        11
       total  |  26.00ms ( 26.00ms)  25.63ms ( 25.63ms)         1
solution: f= 0.3205171991235962
maximized objective =  1.9397871605360368
mean(S) =  3.579256357405445
skew(S) =  0.30031796366661256


## now brute force iterate over all R&D combinations

### request target mean of 3.2

In [10]:
def pareto_plot(solutions_list, labels=None):
    import plotly.graph_objects as go
    colors = ['blue', 'red', 'green', 'orange', 'purple']
    fig = go.Figure()
    if labels is None:
        labels = [f'Set {i+1}' for i in range(len(solutions_list))]
    for idx, solutions in enumerate(solutions_list):
        S_mean_vals = [sol['S_mean'] for sol in solutions]
        S_skew_vals = [sol['S_skew'] for sol in solutions]
        f_vals = [sol['f'] for sol in solutions]
        cnt_r_vals = [sol['cnt_r'] for sol in solutions]
        cnt_d_vals = [sol['cnt_d'] for sol in solutions]
        try:
            alpha_vals = [sol['alpha'] for sol in solutions]
        except Exception as e:
            alpha_vals = None
        hover_text = [
            f"f: {f:.3f}<br>cnt_r: {cnt_r}<br>cnt_d: {cnt_d}<br>alpha: {alpha}"
            for f, cnt_r, cnt_d, alpha in zip(f_vals, cnt_r_vals, cnt_d_vals, alpha_vals if alpha_vals is not None else [None]*len(f_vals))
        ]
        fig.add_trace(go.Scatter(
            x=S_mean_vals,
            y=S_skew_vals,
            mode='markers',
            marker=dict(size=12, color=colors[idx % len(colors)], symbol='circle', line=dict(width=2, color='black')),
            text=hover_text,
            hoverinfo='text',
            name=labels[idx]
        ))
    fig.update_layout(
        title='Pareto Plot: S_mean vs S_skew',
        xaxis_title='S_mean',
        yaxis_title='S_skew',
        showlegend=True
    )
    fig.show()

In [11]:
obj = 'target_mean'
cnt_r=0
cnt_d=0
solutions = []
for R in [R1,R2,R3]:
    cnt_r+=1
    cnt_d=0
    for D in [D1,D2,D3,D4]:
        cnt_d+=1
        try:
            f_opt, S_mean, S_skew = inner_op(R,D,obj,verbose=True)
            solutions.append({
                'f': f_opt,
                'cnt_r': cnt_r,
                'cnt_d': cnt_d,
                'S_mean': S_mean,
                'S_skew': S_skew
            })
        except:
            print('no feasible solution')


pareto_plot([solutions],labels=['target_mean'])
solutions1=solutions

      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |   3.00ms (115.38us)   1.74ms ( 66.85us)        26
       nlp_g  |        0 (       0) 597.00us ( 66.33us)         9
  nlp_grad_f  |   1.00ms (100.00us)   1.10ms (109.60us)        10
  nlp_hess_l  |   2.00ms (250.00us)   1.85ms (231.75us)         8
   nlp_jac_g  |        0 (       0)   1.14ms (113.70us)        10
       total  |   6.00ms (  6.00ms)   6.83ms (  6.83ms)         1
solution: f= 1.0000000067120243
maximized objective =  0.02520543866668551
mean(S) =  3.3587622079296127
skew(S) =  0.43177852728553007
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |   1.00ms ( 50.00us)   1.35ms ( 67.65us)        20
       nlp_g  |        0 (       0) 465.00us ( 66.43us)         7
  nlp_grad_f  |   1.00ms (125.00us) 879.00us (109.87us)         8
  nlp_hess_l  |   2.00ms (333.33us)   1.40ms (233.50us)         6
   nlp_jac_g  |        0 (       0) 912.00us (114.00us)         8
      

"best"  solution selected to reach exactly mean(S) = 3.2 mm/s:
- f = 0.356
- combination: R3 + D4

![](./target_mean.png)

In [12]:
analyze_system_vibration(system_vibration(R3,D4,0.356))

mean: 3.18, skewness: 0.5, std: 0.98


### minimize mean

In [13]:
obj = 'min_mean'
cnt_r=0
cnt_d=0
solutions = []
for R in [R1,R2,R3]:
    cnt_r+=1
    cnt_d=0
    for D in [D1,D2,D3,D4]:
        cnt_d+=1
        try:
            f_opt, S_mean, S_skew = inner_op(R,D,obj,verbose=True)
            solutions.append({
                'f': f_opt,
                'cnt_r': cnt_r,
                'cnt_d': cnt_d,
                'S_mean': S_mean,
                'S_skew': S_skew
            })
        except:
            print('no feasible solution')

pareto_plot([solutions],labels=['min_mean'])
solutions2=solutions

      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |        0 (       0)   2.13ms ( 92.65us)        23
       nlp_g  |        0 (       0) 540.00us ( 67.50us)         8
  nlp_grad_f  |   2.00ms (222.22us) 882.00us ( 98.00us)         9
  nlp_hess_l  |   3.00ms (428.57us)   1.19ms (170.00us)         7
   nlp_jac_g  |   1.00ms (111.11us)   1.14ms (126.78us)         9
       total  |   6.00ms (  6.00ms)   6.35ms (  6.35ms)         1
solution: f= 1.0000000089560142
maximized objective =  3.358762205780144
mean(S) =  3.358762205780144
skew(S) =  0.43177852952585266
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |   1.00ms ( 50.00us)   1.35ms ( 67.35us)        20
       nlp_g  |   1.00ms (142.86us) 472.00us ( 67.43us)         7
  nlp_grad_f  |   1.00ms (125.00us) 670.00us ( 83.75us)         8
  nlp_hess_l  |   1.00ms (166.67us) 994.00us (165.67us)         6
   nlp_jac_g  |        0 (       0) 934.00us (116.75us)         8
       to

"best"  solution selected to reach minimal mean(S):
- f = 0.634
- combination: R3 + D4

![](./min_mean.png)

In [14]:
analyze_system_vibration(system_vibration(R3,D4,0.634))

mean: 2.78, skewness: 1.26, std: 0.91


### minimize mean and skew at once

note: in this case we have 2 objectives as a linear combination in the cost function with weighting factor $\alpha \in [0,1]$

In [15]:
obj = 'min_mean_and_skew'
cnt_r=0
cnt_d=0
solutions = []
for R in [R1,R2,R3]:
    cnt_r+=1
    cnt_d=0
    for D in [D1,D2,D3,D4]:
        cnt_d+=1
        for alpha in np.linspace(0.01,0.99,10):
            try:
                f_opt, S_mean, S_skew = inner_op(R,D,obj,verbose=True,alpha=alpha)
                solutions.append({
                    'f': f_opt,
                    'cnt_r': cnt_r,
                    'cnt_d': cnt_d,
                    'S_mean': S_mean,
                    'S_skew': S_skew,
                    'alpha': alpha
                })
            except:
                print('no feasible solution')


pareto_plot([solutions],labels=['min_mean_and_skew'])
solutions3=solutions

      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |   8.00ms (400.00us)   8.31ms (415.45us)        20
       nlp_g  |        0 (       0) 519.00us ( 74.14us)         7
  nlp_grad_f  |   3.00ms (375.00us)   4.20ms (525.25us)         8
  nlp_hess_l  |   6.00ms (  1.00ms)   5.55ms (924.50us)         6
   nlp_jac_g  |   3.00ms (375.00us)   1.66ms (207.37us)         8
       total  |  21.00ms ( 21.00ms)  21.18ms ( 21.18ms)         1
solution: f= 0.6504822488124902
maximized objective =  0.18952721039107467
mean(S) =  3.8000000347205924
skew(S) =  0.15305778792303948
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |  10.00ms (434.78us)  10.11ms (439.61us)        23
       nlp_g  |   1.00ms (125.00us) 785.00us ( 98.12us)         8
  nlp_grad_f  |   7.00ms (777.78us)   5.58ms (619.56us)         9
  nlp_hess_l  |   7.00ms (  1.00ms)   7.25ms (  1.04ms)         7
   nlp_jac_g  |        0 (       0)   1.47ms (163.00us)         9
      

selected "best" solution to minimize mean and skew:
- f = 0.363
- combination: R3 + D4

![](./min_mean_and_skew.png)

In [19]:
analyze_system_vibration(system_vibration(R3,D4,0.363))

mean: 3.15, skewness: 0.51, std: 0.98


### pareto plot of all 3 versions

In [17]:
pareto_plot([solutions1, solutions2, solutions3], labels=['Target Mean', 'Min Mean', 'Min Mean & Skew'])

# proposed solution by InsideOpt
- f = 0.74
- R3+D1 combination

should have 
- S_mean=3.39 mm/s
- S_skew = 0.353

In [21]:
print('proposed in puzzle solution: ')
analyze_system_vibration(system_vibration(R3,D1,0.74))
print('my closest fit:')
analyze_system_vibration(system_vibration(R3,D1,0.841))

proposed in puzzle solution: 
mean: 3.39, skewness: 0.38, std: 1.02
my closest fit:
mean: 3.27, skewness: 0.47, std: 1.02
